In [6]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [4]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [16]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Eustrat Zhupa',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Borrowing the words of another reviewer here whose comment cannot be topped : Truly an icon'},
 {'professor': 'Elias Mandala',
  'subject': 'History',
  'stars': 5,
  'review': 'Take it, take it, take it! The best History Professor I have seen out there. Always encourages his students to attend office hours, and even asked students to redo their papers if they fall below the class average. An incredible professor he is.'},
 {'professor': 'Nese Yildiz',
  'subject': 'Economics',
  'stars': 2,
  'review': "A sweet and encouraging professor. Classes could better, but hey, let's be nice."},
 {'professor': 'Bonnie Meguid',
  'subject': 'Political Science',
  'stars': 4,
  'review': "I really loved Professor Meguid! She's unbelievably smart and really cares about her students. She is very accessible outside of class and super responsive to emails. You can't BS the readings though so don't make my mist

In [17]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [14]:
processed_data[0]

{'values': [0.04466424509882927,
  -0.0020518240053206682,
  -0.07752101868391037,
  0.053685374557971954,
  0.04703677445650101,
  -0.00011045796418329701,
  -0.037546657025814056,
  0.05539580062031746,
  -0.0067761922255158424,
  -0.029242804273962975,
  -0.026442667469382286,
  -0.005058866925537586,
  0.005072660744190216,
  -0.04176755249500275,
  0.019021615386009216,
  0.021821752190589905,
  -0.0024846037849783897,
  -0.018759533762931824,
  0.04452630504965782,
  0.011248824186623096,
  0.044940121471881866,
  0.006431347690522671,
  -0.01063500065356493,
  0.02339424379169941,
  0.011207442730665207,
  0.02962902933359146,
  -0.035725876688957214,
  0.025394340977072716,
  0.02758754976093769,
  -0.04066405072808266,
  -0.0035932783503085375,
  -0.039919186383485794,
  0.011235030367970467,
  -0.03809840604662895,
  -0.00438297213986516,
  0.02002856135368347,
  0.04364350438117981,
  -0.026801304891705513,
  0.020483756437897682,
  0.012035069055855274,
  0.0123040471225976

In [18]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}